In [13]:
import numpy as np 
import pandas as pd 
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix , classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
import psycoph2
import config as creds

In [14]:
#Set up connection to database
def connect():
    
    # Set up a connection to the postgres server.
    conn_string = "host="+ creds.PGHOST +" port="+ "5432" +" dbname="+ creds.PGDATABASE +" user=" + creds.PGUSER \
                  +" password="+ creds.PGPASSWORD
    
    conn = psycopg2.connect(conn_string)
    print("Connected!")

    # Create a cursor object
    cursor = conn.cursor()
    
    return conn, cursor


Diabetes_binary  HighBP  HighChol  CholCheck   BMI  Smoker  Stroke  \
0              0.0     1.0       0.0        1.0  26.0     0.0     0.0   
1              0.0     1.0       1.0        1.0  26.0     1.0     1.0   
2              0.0     0.0       0.0        1.0  26.0     0.0     0.0   
3              0.0     1.0       1.0        1.0  28.0     1.0     0.0   
4              0.0     0.0       0.0        1.0  29.0     1.0     0.0   

   HeartDiseaseorAttack  PhysActivity  Fruits  ...  AnyHealthcare  \
0                   0.0           1.0     0.0  ...            1.0   
1                   0.0           0.0     1.0  ...            1.0   
2                   0.0           1.0     1.0  ...            1.0   
3                   0.0           1.0     1.0  ...            1.0   
4                   0.0           1.0     1.0  ...            1.0   

   NoDocbcCost  GenHlth  MentHlth  PhysHlth  DiffWalk  Sex   Age  Education  \
0          0.0      3.0       5.0      30.0       0.0  1.0   4.0        6.0   
1          0.0      3.0       0.0       0.0       0.0  1.0  12.0        6.0   
2          0.0      1.0       0.0      10.0       0.0  1.0  13.0        6.0   
3          0.0      3.0       0.0       3.0       0.0  1.0  11.0        6.0   
4          0.0      2.0       0.0       0.0       0.0  0.0   8.0        5.0   

   Income  
0     8.0  
1     8.0  
2     8.0  
3     8.0  
4     8.0  

[5 rows x 22 columns]

In [ ]:
conn, cursor = connect()

In [ ]:
# Load the data

file_path = ('diabetes_012_health_indicators_BRFSS2015.csv')
df = pd.read_csv(file_path)
#print(df.shape)

#Convert diabetes to 1 and prediabetes to 0
def diabetes_binary(x, r):
    if x == r:
        return('1')
    if x < r:
        return('0')
    
df['Diabetes_012'] = df["Diabetes_012"].apply(diabetes_binary, args =[2])



df.head(20)


In [15]:
df = df.dropna()

In [ ]:
#Drops NA rows
df.dropna()

In [16]:
#Split the data into features(x) and Target(y)
y = df["Diabetes_binary"]
X = df.drop(columns="Diabetes_binary")

In [17]:
y.value_counts()

0.0    35346
1.0    35346
Name: Diabetes_binary, dtype: int64

In [18]:
X.describe()

HighBP      HighChol     CholCheck           BMI        Smoker  \
count  70692.000000  70692.000000  70692.000000  70692.000000  70692.000000   
mean       0.563458      0.525703      0.975259     29.856985      0.475273   
std        0.495960      0.499342      0.155336      7.113954      0.499392   
min        0.000000      0.000000      0.000000     12.000000      0.000000   
25%        0.000000      0.000000      1.000000     25.000000      0.000000   
50%        1.000000      1.000000      1.000000     29.000000      0.000000   
75%        1.000000      1.000000      1.000000     33.000000      1.000000   
max        1.000000      1.000000      1.000000     98.000000      1.000000   

             Stroke  HeartDiseaseorAttack  PhysActivity        Fruits  \
count  70692.000000          70692.000000  70692.000000  70692.000000   
mean       0.062171              0.147810      0.703036      0.611795   
std        0.241468              0.354914      0.456924      0.487345   
min        0.000000              0.000000      0.000000      0.000000   
25%        0.000000              0.000000      0.000000      0.000000   
50%        0.000000              0.000000      1.000000      1.000000   
75%        0.000000              0.000000      1.000000      1.000000   
max        1.000000              1.000000      1.000000      1.000000   

            Veggies  ...  AnyHealthcare   NoDocbcCost       GenHlth  \
count  70692.000000  ...   70692.000000  70692.000000  70692.000000   
mean       0.788774  ...       0.954960      0.093914      2.837082   
std        0.408181  ...       0.207394      0.291712      1.113565   
min        0.000000  ...       0.000000      0.000000      1.000000   
25%        1.000000  ...       1.000000      0.000000      2.000000   
50%        1.000000  ...       1.000000      0.000000      3.000000   
75%        1.000000  ...       1.000000      0.000000      4.000000   
max        1.000000  ...       1.000000      1.000000      5.000000   

           MentHlth      PhysHlth      DiffWalk           Sex           Age  \
count  70692.000000  70692.000000  70692.000000  70692.000000  70692.000000   
mean       3.752037      5.810417      0.252730      0.456997      8.584055   
std        8.155627     10.062261      0.434581      0.498151      2.852153   
min        0.000000      0.000000      0.000000      0.000000      1.000000   
25%        0.000000      0.000000      0.000000      0.000000      7.000000   
50%        0.000000      0.000000      0.000000      0.000000      9.000000   
75%        2.000000      6.000000      1.000000      1.000000     11.000000   
max       30.000000     30.000000      1.000000      1.000000     13.000000   

          Education        Income  
count  70692.000000  70692.000000  
mean       4.920953      5.698311  
std        1.029081      2.175196  
min        1.000000      1.000000  
25%        4.000000      4.000000  
50%        5.000000      6.000000  
75%        6.000000      8.000000  
max        6.000000      8.000000  

[8 rows x 21 columns]

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1,)




In [20]:
X_test

HighBP  HighChol  CholCheck   BMI  Smoker  Stroke  \
25002     1.0       0.0        1.0  30.0     0.0     0.0   
45944     1.0       1.0        1.0  33.0     1.0     0.0   
23894     1.0       0.0        1.0  27.0     0.0     0.0   
6931      0.0       0.0        1.0  25.0     1.0     0.0   
39255     0.0       1.0        1.0  29.0     0.0     0.0   
...       ...       ...        ...   ...     ...     ...   
43033     0.0       0.0        1.0  21.0     0.0     0.0   
30157     1.0       1.0        1.0  37.0     1.0     0.0   
13101     1.0       1.0        1.0  31.0     0.0     0.0   
47311     1.0       0.0        1.0  36.0     1.0     0.0   
16611     1.0       1.0        1.0  39.0     1.0     0.0   

       HeartDiseaseorAttack  PhysActivity  Fruits  Veggies  ...  \
25002                   0.0           1.0     1.0      1.0  ...   
45944                   0.0           1.0     0.0      1.0  ...   
23894                   0.0           0.0     0.0      1.0  ...   
6931                    0.0           1.0     1.0      1.0  ...   
39255                   0.0           1.0     1.0      0.0  ...   
...                     ...           ...     ...      ...  ...   
43033                   0.0           1.0     1.0      1.0  ...   
30157                   0.0           1.0     1.0      1.0  ...   
13101                   0.0           1.0     0.0      1.0  ...   
47311                   0.0           1.0     0.0      1.0  ...   
16611                   1.0           1.0     1.0      0.0  ...   

       AnyHealthcare  NoDocbcCost  GenHlth  MentHlth  PhysHlth  DiffWalk  Sex  \
25002            0.0          0.0      1.0       0.0       0.0       0.0  1.0   
45944            1.0          0.0      3.0       0.0       0.0       0.0  1.0   
23894            1.0          0.0      2.0       0.0       0.0       0.0  1.0   
6931             1.0          0.0      1.0       0.0       0.0       0.0  0.0   
39255            1.0          0.0      3.0       0.0       0.0       0.0  1.0   
...              ...          ...      ...       ...       ...       ...  ...   
43033            1.0          0.0      1.0       0.0       0.0       0.0  0.0   
30157            1.0          0.0      4.0       0.0      30.0       1.0  1.0   
13101            1.0          0.0      3.0       0.0       0.0       0.0  0.0   
47311            1.0          0.0      2.0       0.0       0.0       0.0  0.0   
16611            1.0          0.0      3.0      10.0       0.0       1.0  0.0   

        Age  Education  Income  
25002   1.0        5.0     2.0  
45944   4.0        5.0     7.0  
23894   9.0        5.0     6.0  
6931   13.0        6.0     8.0  
39255  11.0        6.0     3.0  
...     ...        ...     ...  
43033   8.0        2.0     1.0  
30157  12.0        5.0     6.0  
13101   8.0        3.0     2.0  
47311  12.0        5.0     5.0  
16611  11.0        4.0     4.0  

[17673 rows x 21 columns]

In [21]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
                                   
b_model = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
b_model = b_model.fit(X_train, y_train)






b_model


BalancedRandomForestClassifier(random_state=1)

In [22]:
# Calculated the balanced accuracy score
predictions = b_model.predict(X_test)
acc_score = balanced_accuracy_score(y_test, predictions)
acc_score

0.7389303987249654

In [23]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, predictions)
cm


array([[6288, 2625],
       [1994, 6766]], dtype=int64)

In [24]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, predictions))


                   pre       rec       spe        f1       geo       iba       sup

        0.0       0.76      0.71      0.77      0.73      0.74      0.54      8913
        1.0       0.72      0.77      0.71      0.75      0.74      0.55      8760

avg / total       0.74      0.74      0.74      0.74      0.74      0.54     17673



In [25]:
# List the features sorted in descending order by feature importance
sorted(zip(b_model.feature_importances_, X.columns), reverse=True)

[(0.16901166110146426, 'BMI'),
 (0.12604514893460492, 'Age'),
 (0.10926745022967654, 'GenHlth'),
 (0.08415574575222536, 'Income'),
 (0.07591284315460026, 'HighBP'),
 (0.06911292538852137, 'PhysHlth'),
 (0.05715631062519446, 'Education'),
 (0.051423287068048495, 'MentHlth'),
 (0.03887029081374438, 'HighChol'),
 (0.028998627141132514, 'Smoker'),
 (0.02861532244575861, 'Fruits'),
 (0.026620071867702294, 'Sex'),
 (0.025156712667624082, 'DiffWalk'),
 (0.023433393337751407, 'PhysActivity'),
 (0.02204586409549268, 'Veggies'),
 (0.018778852072519014, 'HeartDiseaseorAttack'),
 (0.011986287496732863, 'NoDocbcCost'),
 (0.010211391355060422, 'Stroke'),
 (0.009797727163657562, 'HvyAlcoholConsump'),
 (0.007381615442186459, 'AnyHealthcare'),
 (0.006018471846302041, 'CholCheck')]

In [ ]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier
e_model =  EasyEnsembleClassifier(n_estimators=100, random_state=1)
e_model.fit(X_train,y_train)

In [ ]:
# Calculated the balanced accuracy score
e_predictions = e_model.predict(X_test)
e_acc_score = balanced_accuracy_score(y_test, e_predictions)
e_acc_score

In [ ]:
# Display the confusion matrix
cm = confusion_matrix(y_test, e_predictions)
cm

In [ ]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, e_predictions))



In [ ]:
# Execute SQL Command and commit to DB
conn.commit()